<a href="https://colab.research.google.com/github/TonyLiu836/Road-Network-Extraction-And-Route-Travel-Time-Estimation/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/CSCI5525Project"
%pwd

In [ ]:
!pip install geopandas    #==0.6.3   #latest version of geopandas changes CRS objects, 0.6.3 works when generating training masks - otherwise shouldnt matter
!pip install imagecodecs    
!pip install osmnx
!pip install scikit-image

# Data Prep & Training Mask Generation (No Need to Run)

In [ ]:
from PIL import Image   #PIL can't open 16bit images, only 8bit

im = Image.open('/content/drive/MyDrive/CSCI5525Project/SN3_roads_train_AOI_3_Paris_PS-RGB_img10.tif')
im.show()

FileNotFoundError: ignored

In [ ]:
#!unzip /content/drive/MyDrive/CSCI5525Project/paris.zip -d /home

!cp -r paris_PSMS_8bit /home
!cp -r paris_geojson_roads_speed/ /home

cp: cannot stat 'paris_PSMS_8bit': No such file or directory
cp: cannot stat 'paris_geojson_roads_speed/': No such file or directory


In [ ]:
#DONT NEED TO RUN THIS 

#convert 16-bit satellite images into 8-bit for Paris Dataset
!python create_8bit_images.py \
    --indir="/home/paris/AOI_3_Paris/PS-MS" \
    --outdir="/content/drive/MyDrive/CSCI5525Project/paris_PSMS_8bit" \
    --rescale_type=perc \
    --percentile=2,98 \
    --band_order=5,3,2

#create road speed mask for Paris Dataset 
!python speed_masks.py \
    --geojson_dir="/home/paris_geojson_roads_speed" \
    --image_dir="/home/paris_PSMS_8bit" \
    --output_conversion_csv_binned="/content/drive/MyDrive/CSCI5525Project/paris_roads_train_speed_conversion_binned.csv" \
    --output_mask_dir_contin="/content/drive/MyDrive/CSCI5525Project/paris_training_masks_contin" \
    --output_mask_multidim_dir="/content/drive/MyDrive/CSCI5525Project/paris_train_mask_binned_mc" \
    --buffer_distance_meters=2

python3: can't open file 'create_8bit_images.py': [Errno 2] No such file or directory
python3: can't open file 'speed_masks.py': [Errno 2] No such file or directory


# Split Data (Don't need to run again)

In [ ]:
#works but isnt exactl what we want (cant split imgs along with masks and geojsons )
!pip install split-folders  #lets you split data in folders directly into train/test splits
import splitfolders 

#splitfolders.ratio("/content/drive/MyDrive/CSCI5525Project/experimentsplitdata", output="/content/drive/MyDrive/CSCI5525Project/outputsplitdata", seed = 1337, ratio=(0.8,0.2), group_prefix=2)

splitfolders.ratio("/content/drive/MyDrive/CSCI5525Project/experimentsplitdata", output="/content/drive/MyDrive/CSCI5525Project/outputsplitdata", seed = 1337, ratio=(0.8,0.2))


In [ ]:
#*************DO NOT RUN THIS*****************************
#**********TEST VERSION********************
import os
from glob import glob
import shutil
from sklearn.model_selection import train_test_split
import re 

#splits files into train and test (70/30)

def batch_move_files(file_list, AOI, img_path, gjson_path, mask_path, destination_path):
    for file in file_list:
        #extracting only the name of the file and concatenating with extenions
        city =  AOI.split('_')[-1].lower()
        if not os.path.isdir(destination_path +"/"+ city + "_PSMS_8bit"):
            os.mkdir(destination_path +"/"+ city + "_PSMS_8bit")
        if not os.path.isdir(destination_path +"/"+ city + "_geojson"):
            os.mkdir(destination_path +"/"+ city + "_geojson")
        if not os.path.isdir(destination_path +"/"+ city + "_masks"):
            os.mkdir(destination_path +"/"+ city + "_masks")
        
        
        imgNum = re.findall("\d+", file)[-1]
        #print("file", file)
        #image = file.split('\\')[1] + '.tif'
        image = file + '.tif'
        if AOI == "SN5_roads_train_AOI_8_Mumbai":
            gjson = AOI + "_geojson_roads_speed_chip" + str(imgNum)+ ".geojson"
            mask = AOI+"_PS-MS_chip"+str(imgNum)+".tif"
        else:
            gjson = AOI+"_geojson_roads_speed_img" + str(imgNum) + ".geojson"
            #print("gjson", gjson)
            mask = AOI+"_PS-MS_img"+str(imgNum)+".tif"
            #print("mask", mask)
        #xml = file.split('\\')[1] + '.xml'
        #gjson = file.split('\\')[1]+'.geojson'
        #print("image", image)
        #print("os join", os.path.join(img_path, image))
        #print("gjson", gjson)
        shutil.copy(os.path.join(img_path, image), destination_path +"/"+ city + "_PSMS_8bit")
        shutil.copy(os.path.join(gjson_path, gjson), destination_path +"/"+ city + "_geojson")
        shutil.copy(os.path.join(mask_path, mask), destination_path + "/"+ city + "_masks")
    return

#source_dir = "E:/ML/DataSet/"
#test_dir = "E:/ML/test/"
#train_dir = "E:/ML/train/"
#batch_move_files(train_names, source_dir, train_dir)
#batch_move_files(test_names, source_dir, test_dir)
#city = "paris"
#AOI = "SN3_roads_train_AOI_3_Paris"

#getting list of images
#image_files = glob("E:/ML/DataSet/*.jpg")
img_dirs = [ "/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/mumbai/*.tif", "/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/paris/*.tif","/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/shanghai/*.tif"]

#image_files = glob("/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/paris/*.tif")
#print(image_files)
#replacing the extension
#images = [name.replace(".tif","") for name in image_files]

#splitting the dataset
#train_names, test_names = train_test_split(images, test_size=0.2)

AOI = ["SN5_roads_train_AOI_8_Mumbai", "SN3_roads_train_AOI_3_Paris", "SN3_roads_train_AOI_4_Shanghai"]
train_parent_dir = "/content/drive/MyDrive/CSCI5525Project/outputsplitdata/train"
test_parent_dir = "/content/drive/MyDrive/CSCI5525Project/outputsplitdata/test"

for dir in range(len(img_dirs)):
    image_files = glob(img_dirs[dir])
    images = [name.replace(".tif","") for name in image_files]
    #print("imgaes=", images)
    #for region in AOI:
    region = AOI[dir]
    train_names, test_names = train_test_split(images, test_size=0.3)
    print(train_names)
    print(test_names)
    city =  region.split('_')[-1].lower()
    #print("city=", city)
    img_dir = "/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/" + city
    gjson_dir = "/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/" + city + "_geojson"
    mask_dir = "/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/" + city + "_masks"
    #img_destination = ""  
    batch_move_files(train_names, region, img_dir, gjson_dir, mask_dir, train_parent_dir)
    batch_move_files(test_names, region, img_dir,gjson_dir,mask_dir, test_parent_dir)
    #print("reaches here")


['/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/mumbai/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip1', '/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/mumbai/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip3', '/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/mumbai/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip4']
['/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/mumbai/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip0', '/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/mumbai/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip2']
['/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/paris/SN3_roads_train_AOI_3_Paris_PS-MS_img10', '/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/paris/SN3_roads_train_AOI_3_Paris_PS-MS_img30', '/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/paris/SN3_roads_train_AOI_3_Paris_PS-MS_img20', '/content/drive/MyDrive/CSCI5525Project/experimentsplitdata/paris/SN3_roads_train_AOI_3_Paris_PS-MS_img19', '/content/drive

In [ ]:

#!mkdir /home/completedata/
#!cp -r /content/drive/MyDrive/CSCI5525Project/paris/paris_PSMS_8bit /home/completedata
#!cp -r /content/drive/MyDrive/CSCI5525Project/paris/paris_geojson_roads_speed /home/completedata
#!cp -r /content/drive/MyDrive/CSCI5525Project/paris/paris_train_mask_binned_mc /home/completedata

#!unzip /content/drive/MyDrive/CSCI5525Project/shanghai/shanghai_PS-RGB_8bit.zip -d /home/completedata

#!unzip /content/drive/MyDrive/CSCI5525Project/mumbai/mumbai_PS-RGB_8bit.zip -d /home/completedata

#!!cp -r /content/drive/MyDrive/CSCI5525Project/mumbai/mumbai_geojson_roads_speed /home/completedata
#!cp -r /content/drive/MyDrive/CSCI5525Project/mumbai/mumbai_train_mask_binned_mc /home/completedata

#!cp -r /content/drive/MyDrive/CSCI5525Project/shanghai/shanghai_geojson_roads_speed /home/completedata
#!cp -r /content/drive/MyDrive/CSCI5525Project/shanghai/shanghai_train_mask_binned_mc /home/completedata



In [ ]:
import os
from glob import glob
import shutil
from sklearn.model_selection import train_test_split
import re 

#splits imgs and corresponding masks/geojson files into train and test (70/30)

def batch_move_files(file_list, AOI, img_path, gjson_path, mask_path, destination_path):
    for file in file_list:
        #extracting only the name of the file and concatenating with extenions
        city =  AOI.split('_')[-1].lower()
        if not os.path.isdir(destination_path +"/"+ city + "_PSMS_8bit"):
            os.mkdir(destination_path +"/"+ city + "_PSMS_8bit")
        if not os.path.isdir(destination_path +"/"+ city + "_geojson"):
            os.mkdir(destination_path +"/"+ city + "_geojson")
        if not os.path.isdir(destination_path +"/"+ city + "_masks"):
            os.mkdir(destination_path +"/"+ city + "_masks")
        
        imgNum = re.findall("\d+", file)[-1]
        image = file + '.tif'
        if AOI == "SN5_roads_train_AOI_8_Mumbai":
            gjson = AOI + "_geojson_roads_speed_chip" + str(imgNum)+ ".geojson"
            mask = AOI+"_PS-MS_chip"+str(imgNum)+".tif"
        else:
            gjson = AOI+"_geojson_roads_speed_img" + str(imgNum) + ".geojson"
            mask = AOI+"_PS-MS_img"+str(imgNum)+".tif"
        shutil.copy(os.path.join(img_path, image), destination_path +"/"+ city + "_PSMS_8bit")
        if os.path.exists(os.path.join(gjson_path, gjson)):
            shutil.copy(os.path.join(gjson_path, gjson), destination_path +"/"+ city + "_geojson")
        shutil.copy(os.path.join(mask_path, mask), destination_path + "/"+ city + "_masks")
    return


img_dirs = [ "/home/completedata/mumbai_PSMS_8bit/*.tif", "/home/completedata/paris_PSMS_8bit/*.tif","/home/completedata/shanghai_PSMS_8bit/*.tif"]

AOI = ["SN5_roads_train_AOI_8_Mumbai", "SN3_roads_train_AOI_3_Paris", "SN3_roads_train_AOI_4_Shanghai"]
train_parent_dir = "/content/drive/MyDrive/CSCI5525Project/train_data"
test_parent_dir = "/content/drive/MyDrive/CSCI5525Project/test_data"

for dir in range(len(img_dirs)):
    image_files = glob(img_dirs[dir])
    images = [name.replace(".tif","") for name in image_files]
    region = AOI[dir]
    train_names, test_names = train_test_split(images, test_size=0.3)
    city =  region.split('_')[-1].lower()
    img_dir = "/home/completedata/" + city + "_PSMS_8bit"
    gjson_dir = "/home/completedata/" + city + "_geojson_roads_speed"
    mask_dir = "/home/completedata/" + city + "_train_mask_binned_mc"
    batch_move_files(train_names, region, img_dir, gjson_dir, mask_dir, train_parent_dir)
    batch_move_files(test_names, region, img_dir,gjson_dir,mask_dir, test_parent_dir)


['/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip955', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip311', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip819', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip1006', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip347', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip854', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip438', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip245', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip999', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip145', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip412', '/home/completedata/mumbai_PSMS_8bit/SN5_roads_train_AOI_8_Mumbai_PS-MS_chip689', '/home/complet